## HashMap

**1. HashMap是线程不同步的，如果需要让HashMap保持线程同步，需要要外边加锁或者synchronize或者是使用**
```java
Map m = Collections.synchronizedMap(new HashMap(...));
```

2. HashMap和ArrayList一样，当迭代器创建后，如果通过迭代器以外的方式修改了HashMap的结构，HashMap会快速失败：throw ConcurrentModificationException   



**快速失败和安全失败**  
 java.util 包的集合类就都是快速失败的；而 java.util.concurrent 包下的类都是安全失败：
 
 快速失败（fail—fast）
在用迭代器遍历一个集合对象时，如果遍历过程中对集合对象的内容进行了修改（增加、删除、修改），则会抛出 Concurrent Modification Exception。

原理：迭代器在遍历时直接访问集合中的内容，并且在遍历过程中使用一个 modCount 变量。集合在被遍历期间如果内容发生变化，就会改变 modCount 的值。每当迭代器使用 hashNext()/next() 遍历下一个元素之前，都会检测 modCount 变量是否为 expectedmodCount 值，是的话就返回遍历；否则抛出异常，终止遍历。

>注意：这里异常的抛出条件是检测到 modCount != expectedmodCount 这个条件。如果集合发生变化时修改 modCount 值刚好又设置为了 expectedmodCount 值，则异常不会抛出。因此，不能依赖于这个异常是否抛出而进行并发操作的编程，这个异常只建议用于检测并发修改的 bug。

>场景：java.util 包下的集合类都是快速失败的，不能在多线程下发生并发修改（迭代过程中被修改）。

二、安全失败（fail—safe）
采用安全失败机制的集合容器，在遍历时不是直接在集合内容上访问的，而是先复制原有集合内容，在拷贝的集合上进行遍历。

原理：由于迭代时是对原集合的拷贝进行遍历，所以在遍历过程中对原集合所作的修改并不能被迭代器检测到，所以不会触发 Concurrent Modification Exception。

>缺点：基于拷贝内容的优点是避免了 Concurrent Modification Exception，但同样地，迭代器并不能访问到修改后的内容，即：迭代器遍历的是开始遍历那一刻拿到的集合拷贝，在遍历期间原集合发生的修改迭代器是不知道的。

>场景：java.util.concurrent 包下的容器都是安全失败，可以在多线程下并发使用，并发修改。

**3. HashMap的存储方式**  <br/><br/>
    HashMap一开始是用**数组链表**存储的，当HashMap存储的数据量达到一定时，存储方式会转换为**红黑树**的方式进行存储
    
用数组来映射HashMap的存储，用链表来解决Hash冲突  

当HashMap链表中元素个数大于等于8时，将链表存储改成红黑树存储；当桶中的元素个数小于等于6时，树的结构还原成链表结构；  
**【注】当链表长度大于8并且数组长度大于64链表才会转为红黑树，不然只是会触发扩容，并不会转换**

**HashMap转换阈值为什么一个是8一个是6**  

可以有效防止链表和树之间的频繁转换；树和链表的频繁转换会降低效率；

**4. HashMap默认容量是16，这个容量必须是2的正数次方**  

理由: HashMap在计算hash的时候，确定落在数组的位置的时候，计算方法是(n - 1) & hash
    1. 减少了hash碰撞，使得散列更加均匀；
    2.(n - 1) & hash，当n为2次幂时，会满足一个公式：(n - 1) & hash = hash % n，计算更加高效。

**5. HashMap的put方法**  
put方法的流程：  

    1. 执行 hash(Object key) 得到一个int类型的hash值,然后根据这个hash值就可以找到Node节点的位置了;

    2. **判断table是否为空**，为空表明这是第一个元素插入，则使用resize()进行扩容，初始大小默认16;（resize()方法可以初始化容量也可以用来扩容）

    3. 如果table为空,说明没有产生**hash碰撞**,则直接插入node节点,调到第5步,否则进行下一步;

    4. 如果table不为空,进行下面三种判断:  
        1.桶中第一个元素(数组中的结点)的hash值相等，key相等,赋值给node节点e;  
        2.hash值不相等,说明key不相等,为红黑树,放入树中,赋值给node节点e;  
        3.hash值不相等,说明key不相等,为链表,遍历链表,赋值给最后一个节点,并返回node节点e(这里面还有长度的判断,当链表长度大于8之后需要将链表转化为红黑树进行存储);  

    5. 判断新插入这个值是否导致**size已经超过了阈值**，是则进行扩容


    总结：
    HashMap在put方法中，它使用hashCode()和equals()方法。当我们通过传递key-value对调用put方法的时候，HashMap使用Key hashCode()和哈希算法来找出存储key-value对的索引。如果索引处为空，则直接插入到对应的数组中，否则，判断是否是红黑树，若是，则红黑树插入，否则遍历链表，若长度不小于8，则将链表转为红黑树，转成功之后 再插入。

**6. HashMap的扩容**   
双倍扩容法  
扩容后元素的位置要重新has  
重新hash后的位置：元素的位置要么在原来的位置，要么在 原来索引+扩容前数组长度

**7. HashMap和Hashtable**  
1. 底层数据结构不同: jdk1.7底层都是数组+链表,但jdk1.8 HashMap加入了红黑树
2. Hashtable 是不允许键或值为 null 的，HashMap 的键值则都可以为 null。
3. 添加key-value的hash值算法不同：HashMap添加元素时，是使用自定义的哈希算法,而HashTable是直接采用key的hashCode()
4. 实现方式不同：Hashtable 继承的是 Dictionary类，而 HashMap 继承的是 AbstractMap 类。
5. 初始化容量不同：HashMap 的初始容量为：16，Hashtable 初始容量为：11，两者的负载因子默认都是：0.75。
6. 扩容机制不同：当已用容量>总容量 * 负载因子时，HashMap 扩容规则为当前容量翻倍 （2的倍数），Hashtable 扩容规则为当前容量翻倍 +1。
7. 支持的遍历种类不同：HashMap只支持Iterator遍历,而HashTable支持Iterator和Enumeration两种方式遍历
8. 迭代器不同：HashMap的迭代器(Iterator)是fail-fast迭代器，而Hashtable的enumerator迭代器不是fail-fast的。所以当有其它线程改变了HashMap的结构（增加或者移除元素），将会抛出ConcurrentModificationException，但迭代器本身的remove()方法移除元素则不会抛出ConcurrentModificationException异常。但这并不是一个一定发生的行为，要看JVM。而Hashtable 则不会。
9. 部分API不同：HashMap不支持contains(Object value)方法，没有重写toString()方法,而HashTable支持contains(Object value)方法，而且重写了toString()方法
10. 同步性不同: Hashtable是同步(synchronized)的，适用于多线程环境,
而hashmap不是同步的，适用于单线程环境。多个线程可以共享一个Hashtable；而如果没有正确的同步的话，多个线程是不能共享HashMap的。

**8. HashMap的hash()**

```java
static final int hash(Object key) {
        int h;
        return (key == null) ? 0 : (h = key.hashCode()) ^ (h >>> 16);
    }
```

计算key的哈希在调用native的hashCode之后还需要右移16位，理由是：
1. 让高16位也参与运算，让哈希更加均匀，减少哈希冲突

## ConcurrentHashMap

1. 利用 ==CAS + synchronized== 来保证并发更新的安全;   底层使用==数组+链表+红黑树==来实现

2. get() 和 initTable()是没有加锁的，用的是volatile关键字，保证修改后对线程可见  
get() 是Node的成员val是用volatile修饰的和数组用volatile修饰  
    Node 的val 和 next用volatile修饰让 修改对线程可见；  
    用volatile修饰整个Node是为了让 Node 在扩容时 对线程可见
    
在 initTable时，有一个关键的属性sizeCtl  
* sizeCtl ：默认为0，用来控制table的初始化和扩容操作
* -1 代表table正在初始化
* -N 表示有N-1个线程正在进行扩容操作
* 其余情况：
* 1、如果table未初始化，表示table需要初始化的大小。
* 2、如果table初始化完成，表示table的容量，默认是table大小的0.75倍，居然用这个公式算0.75（n - (n >>> 2)）  
因此如果并发时发现sizeCtl < 0 说明已经有线程在给扩容了，这个线程就会调用Thread.yield()让出一次CPU执行时间